In [1]:
import ee
from ipyleaflet import TileLayer

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='gee-project-461505')

#### 대상지 데이터 불러오기

In [2]:
# GEE에 업로드한 데이터 이름
assetName = "projects/gee-project-461505/assets/buffer_4326_3km"
features = ee.FeatureCollection(assetName)

In [4]:
# 구름제거함수
def mask_s2_clouds(image):
    ## TODO 매직넘버 의미 확인 필요
    cloudProb = image.select('MSK_CLDPRB')
    snowProb = image.select('MSK_SNWPRB')
    cloud = cloudProb.lt(10)
    scl = image.select('SCL')
    shadow = scl.eq(3)
    cirrus = scl.eq(10)

    mask = cloud.And(cirrus.neq(1)).And(shadow.neq(1))

    return (image.updateMask(mask)
            .copyProperties(image, ["system:time_start"]))

In [5]:
# 위성의 지수들로 NDVI, EVI 계산 후 추가
def add_variables(image):
    # Compute time in fractional years since the epoch.
    date = ee.Date(image.get('system:time_start'))
    years = date.difference(ee.Date('2016-01-01'), 'year')

    # Add NDVI band
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI').float()

    # Add EVI band
    evi = image.expression(
        '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
        {
            'B8': image.select('B8'),
            'B4': image.select('B4'),
            'B2': image.select('B2')
        }
    ).rename('EVI').float()

    # Add time band
    t = ee.Image(years).rename('t').float()

    # Add constant band
    constant = ee.Image.constant(1)

    # Return the image with the added bands
    return image.addBands([ndvi, evi, t, constant])

In [6]:
# 시간을 새로운 밴드에 붙임
def createTimeBand(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(2015)
    year_band = ee.Image(year).byte().rename('year')
    return year_band.addBands(img)

#### 분석 시작

In [12]:
# 분석 기간
startDate = ee.Date('2024-01-01')
endDate   = ee.Date('2024-12-31')
window    = 1  # months (월별)

collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(features)
    .filterDate(startDate, endDate)  # 원하는 기간
    .map(mask_s2_clouds)
    .map(add_variables))

# 윈도우 시퀀스
nWindows = endDate.difference(startDate,'month').divide(window).toInt()
sequence = ee.List.sequence(0, nWindows)

# 윈도우 합성
def make_window(num):
    num = ee.Number(num)
    win_start = startDate.advance(num.multiply(window),'month')
    win_end   = startDate.advance(num.add(1).multiply(window),'month')

    subset = collection.filterDate(win_start, win_end).select(['NDVI','EVI'])

    # 월별 NDVI/EVI → max 합성
    win_img = subset.max().set({
        'window_start': win_start.format('YYYY-MM'),
        'window_end': win_end.format('YYYY-MM')
    })
    return win_img

monthly_collection = ee.ImageCollection(sequence.map(make_window))

# (월별 NDVI/EVI 통계치)
def zonal_stats(image):
    start = image.get('window_start')
    end   = image.get('window_end')

    fc = image.reduceRegions(
        collection=features,
        reducer=ee.Reducer.mean() \
                    .combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True) \
                    .combine(reducer2=ee.Reducer.minMax(), sharedInputs=True),
        scale=10
    )
    return fc.map(lambda f: f.set({
        'window_start': start,
        'window_end': end
    }))

# ---------------------------
# 지정된 기간 동안 월별 max 이미지들을 다시 통계 처리
period_mean = monthly_collection.mean().toFloat().rename(['NDVI_mean', 'EVI_mean'])
period_med  = monthly_collection.median().toFloat().rename(['NDVI_median', 'EVI_median'])
period_min  = monthly_collection.min().toFloat().rename(['NDVI_min', 'EVI_min'])
period_max  = monthly_collection.max().toFloat().rename(['NDVI_max', 'EVI_max'])
period_std  = monthly_collection.reduce(ee.Reducer.stdDev()).toFloat().rename(['NDVI_stdDev', 'EVI_stdDev'])

# 하나로 합치기
period_stats = (period_mean
                .addBands(period_min)
                .addBands(period_max)
                .addBands(period_std))

period_stats_clipped = period_stats.clip(features)

# 내보내기 (GeoTIFF)
task = ee.batch.Export.image.toDrive(
    image=period_stats_clipped,
    description='NDVI_EVI_period_stats',
    folder='GEE_exports',
    fileNamePrefix='ndvi_evi_period_stats_buffer',
    region=features.geometry(),
    scale=10,
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    maxPixels=1e13,
    formatOptions={'cloudOptimized': True}
)
task.start()